# Experimento TCC

## Libs

In [1]:
from sklearn.neighbors import (
    NeighborhoodComponentsAnalysis, KNeighborsClassifier
)
from psutil import cpu_count
from dotenv import dotenv_values
import numpy as np
import pandas as pd

In [2]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO
from modules.utils import run_experiments, under_sampling, send_email

In [10]:
import os  
os.makedirs('results', exist_ok=True)
os.makedirs('graphs', exist_ok=True)

In [3]:
columns_names = [
    'tau1','tau2','tau3',
    'tau4','p1','p2','p3',
    'p4','g1','g2','g3','g4'
]
dtypes = {i:float for i in columns_names}

EletricalGrid = pd.read_csv(
    'dataset/Data_for_UCI_named.csv',
    dtype=dtypes)

if 'stab' in EletricalGrid.columns:
    EletricalGrid = EletricalGrid.drop('stab', axis=1)

In [4]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [5]:
K_Folds = 5
K_neighbors = 50
max_iter= 100
N_cpu = cpu_count(logical=False)

In [7]:
first_msg = f'''
{'='*50}
Execução de validação cruzada para NCA+PSO, NCA+Gradient e KNN
{'='*50}
Dimensões de X = {X.shape}
KFold = {K_Folds}
Número de vizinhos usados na classificação = {K_neighbors}
Número de núcleos sendo utilizados = {N_cpu}
'''
print(first_msg)


Execução de validação cruzada para NCA+PSO, NCA+Gradient e KNN
Dimensões de X = (300, 12)
KFold = 5
Número de vizinhos usados na classificação = 50
Número de núcleos sendo utilizados = 2



In [ ]:
for i in range(10)
    experiments_configs = {
        'K_Folds':K_Folds,
        'nca_pso':{
            'X':X, 'y':y, 'K_Folds':K_Folds, 'model_class':NCA,
            'model_options':{
                'optimization_func':PSO, 'max_iter':max_iter, 
                'k':K_neighbors, 'cpu_count': N_cpu,
                'swarm_size':10
            }
        },
    #'nca_gradient': {
    #    'X': np.asarray(X), 'y': np.asarray(y), 
    #    'K_Folds': K_Folds,'k': K_neighbors, 
    #    'model_class': NeighborhoodComponentsAnalysis,
    #    'model_options': {
    #        'n_components':None, 'max_iter':max_iter,
    #        'random_state':42
    #    }
    #},
    #'knn': {
    #    'X': np.asarray(X), 'y': np.asarray(y), 
    #    'K_Folds': K_Folds, 
    #    'model_options': {
    #        'n_neighbors': K_neighbors, 'n_jobs': N_cpu
    #    },
    #    'model_class': KNeighborsClassifier
    #}
    }
    df = run_experiments(experiments_configs, n_runs = 1)
    df.to_csv(f'results/NCA+PSO_10_{max_iter}.csv')
    max_iter += 100

In [ ]:
creds = dotenv_values()
send_email('teste', creds)